In [6]:
using CSV
using DataFrames
using PrettyTables
using StatsBase
include("./advgames.jl")

initialize (generic function with 1 method)

In [7]:
triv = Dict(
    (:C, :C) => 0.0,
    (:C, :D) => 0.0,
    (:D, :C) => 0.0,
    (:D, :D) => 0.0
    )

coop = Dict(
    (:C, :C) => 1.0,
    (:C, :D) => 0.0,
    (:D, :C) => 0.0,
    (:D, :D) => 1.0
    )

hilo = Dict(
    (:C, :C) => 3.0,
    (:C, :D) => 0.0,
    (:D, :C) => 0.0,
    (:D, :D) => 1.0
    )

take_turns = Dict(
        (:C, :C) => 1.0,
        (:C, :D) => 3.0,
        (:D, :C) => 2.0,
        (:D, :D) => 0.0
)

sh = Dict(
        (:C, :C) => 3.0,
        (:C, :D) => 0.0,
        (:D, :C) => 2.0,
        (:D, :D) => 1.0
)

pd = Dict(
        (:C, :C) => 3.0,
        (:C, :D) => 0.0,
        (:D, :C) => 4.0,
        (:D, :D) => 1.0
        )
    
games = Dict(
            triv => "triv", 
            coop => "coop",
            hilo => "hilo",
            take_turns => "take_turns",
            sh => "sh",
            pd => "pd"
        )
for (key, value) in games
    println("Key: ", key, " - Value: ", value)
end
        

Key: Dict((:D, :D) => 0.0, (:C, :D) => 0.0, (:C, :C) => 0.0, (:D, :C) => 0.0) - Value: triv
Key: Dict((:D, :D) => 1.0, (:C, :D) => 0.0, (:C, :C) => 3.0, (:D, :C) => 0.0) - Value: hilo
Key: Dict((:D, :D) => 1.0, (:C, :D) => 0.0, (:C, :C) => 1.0, (:D, :C) => 0.0) - Value: coop
Key: Dict((:D, :D) => 1.0, (:C, :D) => 0.0, (:C, :C) => 3.0, (:D, :C) => 2.0) - Value: sh
Key: Dict((:D, :D) => 0.0, (:C, :D) => 3.0, (:C, :C) => 1.0, (:D, :C) => 2.0) - Value: take_turns
Key: Dict((:D, :D) => 1.0, (:C, :D) => 0.0, (:C, :C) => 3.0, (:D, :C) => 4.0) - Value: pd


In [9]:
# Define special data to record

adv_coop(player) = player.adv_type == :C
adv_defect(player) = player.adv_type == :D
should_c(player) = est_move_coop(player) > est_move_defect(player)
should_d(player) = est_move_coop(player) < est_move_defect(player)
should_adv_c(player) = est_adv_coop(player) > est_adv_defect(player)
should_adv_d(player) = est_adv_coop(player) < est_adv_defect(player)

function cum_avg(a::Vector{Float64})  # helper function that returns vector of cumulative averages of vector a
      ca=Float64[]    
      for i in 1:length(a)
          push!( ca, sum(first(a,i))/i )
      end
return ca
end


#individual_incls(model) = [pr_coop(p) for p in allagents(model)]
avg_incl(model) = sum([pr_coop(p) for p in allagents(model)]) / nagents(model)  
 
pr_max_coop(model) = sum([pr_coop(p) == 1.0 for p in allagents(model)]) / nagents(model)
pr_max_defect(model) = sum([pr_coop(p) == 0.0 for p in allagents(model)]) / nagents(model)

cooperate_state(model) = pr_max_coop(model)>.75
defect_state(model) = pr_max_defect(model) >.75

pr_adv_coop(model) = sum([advocate(p) == :C for p in allagents(model)]) / nagents(model)
pr_adv_defect(model) = sum([advocate(p) == :D for p in allagents(model)]) / nagents(model)

pr_hyp_adv_c_play_d(model) = sum([advocate(p) == :C && pr_coop(p) <= 0.15 for p in allagents(model)]) / nagents(model)
pr_hyp_adv_d_play_c(model) = sum([advocate(p) == :D && pr_coop(p) >= 0.85 for p in allagents(model)]) / nagents(model)

avg_pr_max_coop(model) = mean([pr_coop(p) for p in allagents(model)])  >= 0.85
avg_pr_max_defect(model) = mean([pr_coop(p) for p in allagents(model)])  <= 0.15

pr_mistaken_adv(model) = sum([(advocate(p) == :C && est_move_coop(p) < est_move_defect(p)) ||
                              (advocate(p) == :D && est_move_coop(p) > est_move_defect(p))
                            for p in allagents(model)]) / nagents(model)

proportion_should_c(model) = sum([should_c(p) for p in allagents(model)]) / nagents(model)
proportion_should_d(model) = sum([should_d(p) for p in allagents(model)]) / nagents(model)
proportion_should_adv_c(model) = sum([should_adv_c(p) for p in allagents(model)]) / nagents(model)
proportion_should_adv_d(model) = sum([should_adv_d(p) for p in allagents(model)]) / nagents(model)
cc(model) = float(model.payouts[:C, :C])
cd(model) = float(model.payouts[:C, :D])
dc(model) = float(model.payouts[:D, :C])
dd(model) = float(model.payouts[:D, :D])

is_pd(model) = dc(model) > cc(model) > dd(model) > cd(model)
is_chicken(model) = dc(model) > cc(model) && cd(model) > dd(model)
is_sh(model) = cc(model) > dc(model) && dd(model) > cd(model) && cc(model) > dd(model)
is_c_dom(model) = cc(model) > dc(model) && cd(model) > dd(model) && cc(model) > dd(model)
is_d_dom(model) = dd(model) > cd(model) && dc(model) > cc(model) && dd(model) > cc(model)

function game_type(model) 
    if is_pd(model)
        return "PD"
    elseif is_chicken(model)
        return "CH"
    elseif is_sh(model)
        return "SH"
    elseif is_c_dom(model)
        return "Cdom"
    elseif is_d_dom(model)
        return "Ddom"
    else 
        return "Other"
    end
end

cooperative(player) = pr_coop(player) >= (1-player.move_increment)
uncooperative(player) = pr_coop(player) <= player.move_increment

perc_cooperative(model) = sum([cooperative(p) for p in allagents(model)]) / nagents(model)

perc_uncooperative(model) = sum([uncooperative(p) for p in allagents(model)]) / nagents(model)

perc_middle(model) = sum([!cooperative(p) && !uncooperative(p) for p in allagents(model)]) / nagents(model)

variance_inclinations(model) = var([pr_coop(p) for p in allagents(model)])



variance_inclinations (generic function with 1 method)

In [10]:
model, parameters = initialize(    
    num_cooperators = 6, 
    num_defectors = 6, 
    num_neutral = 0, 
    max_neg_pressure = 4,
    adv_mutation_rate = 0.2,
    move_mutation_rate = 0.15,
    tolerance = 0.05,
    payouts = Dict(
        (:C, :C) => 3.0,
        (:C, :D) => 0.0,
        (:D, :C) => 5.0,
        (:D, :D) => 1.0),
)

pretty_table(parameters, show_subheader=false)
# write the parameters to a txt file
open("analysis_parameters.txt", "w") do io
    pretty_table(io, parameters, show_subheader=false)
end

┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


In [6]:
numrounds=1_000_000

adata = [pr_coop]

# model data to record
mdata = [
    cc, 
    cd, 
    dc, 
    dd, 
    game_type,
    avg_incl,
    pr_adv_coop,
    pr_hyp_adv_c_play_d, 
    pr_hyp_adv_d_play_c, 
    perc_cooperative,
    perc_uncooperative,
    perc_middle,
    variance_inclinations
    ]



13-element Vector{Function}:
 cc (generic function with 1 method)
 cd (generic function with 1 method)
 dc (generic function with 1 method)
 dd (generic function with 1 method)
 game_type (generic function with 1 method)
 avg_incl (generic function with 1 method)
 pr_adv_coop (generic function with 1 method)
 pr_hyp_adv_c_play_d (generic function with 1 method)
 pr_hyp_adv_d_play_c (generic function with 1 method)
 perc_cooperative (generic function with 1 method)
 perc_uncooperative (generic function with 1 method)
 perc_middle (generic function with 1 method)
 variance_inclinations (generic function with 1 method)

In [7]:


for num in 1:50
    model, parameters = initialize(    
        num_cooperators = 6, 
        num_defectors = 6, 
        num_neutral = 0, 
        max_neg_pressure = 4,
        adv_mutation_rate = 0.2,
        move_mutation_rate = 0.15,
        tolerance = 0.05,
            payouts = Dict(
            (:C, :C) => 3.0,
            (:C, :D) => 0.0,
            (:D, :C) => 5.0,
            (:D, :D) => 1.0),
    )
    
    pretty_table(parameters, show_subheader=false)
    # write the parameters to a txt file
    open("multiple_runs/parameters_$(num).txt", "w") do io
        pretty_table(io, parameters, show_subheader=false)
    end

    _, mdf = run!(model, dummystep, model_step!, numrounds; adata, mdata)

    CSV.write("multiple_runs/multiple_runs_$(num).csv", mdf)

    println("finished...")

end



┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...


┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


finished...
┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...


┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


finished...
┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...

...

finished...
┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...


┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


finished...
┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...


┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


finished...


In [39]:
initial_conditions = [
    (6, 6, 0), 
    (6, 0, 6), 
    (0, 6, 6), 
    (4, 4, 4),
    (12, 0, 0), 
    (0, 12, 0), 
    (0, 0, 12)
]

for (num_cooperators, num_defectors, num_neutral) in initial_conditions
    model, parameters = initialize(    
        num_cooperators = num_cooperators, 
        num_defectors = num_defectors, 
        num_neutral = num_neutral, 
        max_neg_pressure = 4,
        adv_mutation_rate = 0.2,
        move_mutation_rate = 0.15,
        tolerance = 0.05,
            payouts = Dict(
            (:C, :C) => 3.0,
            (:C, :D) => 0.0,
            (:D, :C) => 5.0,
            (:D, :D) => 1.0),
    )
    
    pretty_table(parameters, show_subheader=false)
    # write the parameters to a txt file
    open("initial_conditions/parameters_$(num_cooperators)_$(num_defectors)_$(num_neutral).txt", "w") do io
        pretty_table(io, parameters, show_subheader=false)
    end

    _, mdf = run!(model, dummystep, model_step!, numrounds; adata, mdata)

    CSV.write("initial_conditions/initial_conditions_$(num_cooperators)_$(num_defectors)_$(num_neutral).csv", mdf)

    println("finished...")

end



┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...


┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    0 │
│          num_cooperators │                    6 │
│              num_neutral │                    6 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


finished...
┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    0 │
│              num_neutral │                    6 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...


┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    4 │
│          num_cooperators │                    4 │
│              num_neutral │                    4 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


finished...
┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    0 │
│          num_cooperators │                   12 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...


┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                   12 │
│          num_cooperators │                    0 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


finished...
┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    0 │
│          num_cooperators │                    0 │
│              num_neutral │                   12 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...

In [40]:
adv_reassess_frequencies = [
    0.0, 
    0.03, 
    0.06, 
    0.1,
    0.2,
    0.5,
    1.0 
]

for adv_reassess_frequency in adv_reassess_frequencies
    model, parameters = initialize(    
        num_cooperators = 6, 
        num_defectors = 6, 
        num_neutral = 0, 
        max_neg_pressure = 4,
        adv_mutation_rate = 0.2,
        move_mutation_rate = 0.15,
        adv_reassess_frequency = adv_reassess_frequency,
        tolerance = 0.05,
            payouts = Dict(
            (:C, :C) => 3.0,
            (:C, :D) => 0.0,
            (:D, :C) => 5.0,
            (:D, :D) => 1.0),
    )
    
    pretty_table(parameters, show_subheader=false)
    # write the parameters to a txt file
    open("adv_reassess_frequencies/parameters_$(adv_reassess_frequency).txt", "w") do io
        pretty_table(io, parameters, show_subheader=false)
    end

    _, mdf = run!(model, dummystep, model_step!, numrounds; adata, mdata)

    CSV.write("adv_reassess_frequencies/advocate_reassess_$(adv_reassess_frequency).csv", mdf)

    println("finished...")

end



┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │           [0.0, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...


┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.03, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


finished...
┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │          [0.06, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...


┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │           [0.1, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


finished...
┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │           [0.2, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...


┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │           [0.5, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘


finished...
┌──────────────────────────┬──────────────────────┐
│                     Keys │               Values │
├──────────────────────────┼──────────────────────┤
│             base_payouts │ [5.0, 3.0, 1.0, 0.0] │
│                tolerance │                 0.05 │
│            num_defectors │                    6 │
│          num_cooperators │                    6 │
│              num_neutral │                    0 │
│    max_pos_neg_pressures │               [6, 4] │
│               population │                   12 │
│       a_m_mutation_rates │          [0.2, 0.15] │
│   a_m_weights_of_present │           [0.3, 0.2] │
│ a_m_reassess_frequencies │           [1.0, 0.1] │
│              m_increment │                 0.15 │
└──────────────────────────┴──────────────────────┘
finished...